# Module 2. Local Inference

***[주의] 본 노트북은 Python 2가 아닌 Python 3에서 실행하셔야 합니다. 배치 데이터 추론 결과에 대한 다중 클래스 분류(multi-class classification) 지표(metric) 계산 시 scikit-learn 0.23.1 버전 기준의 빌트인 함수를 사용하는데, scikit-learn이 0.20 버전 이후로는 Python 2를 지원하지 않기 때문입니다.***


## Introduction

이 튜토리얼에서는 이전 노트북에서 Amazon SageMaker의 Tensorflow 1.x 사용자 스크립트로 학습한 MobileNet 모델 아티팩트 중 네트워크 구조와 가중치를 바이너리 포맷(protobuf)으로 저장한 `frozen.pb` 파일을 사용하여 로컬 상에서 추론을 수행합니다. 
이를 통해 아래의 작업들을 쉽고 빠르게 수행할 수 있습니다.

- TFLite 변환 전, 테스트셋 상에서 모델 추론 성능의 빠른 검증
- TFLite 변환 전/후 성능 편차 테스트
- 검증 데이터 및 테스트 데이터에 대해 추론 결과가 좋지 않은 경우들(예: 오분류, 미검출, 낮은 예측 score 등)에 대해 Amazon Augmented AI(이하 A2I)를 사용하여 모델 성능 개선

물론 추론 컨테이너 환경을 구축하여 SageMaker 엔드포인트 배포하여 추론을 수행할 수도 있지만, 실습의 편의성을 위해 별도의 추론 컨테이너 없이 곧바로 네트워크 구조와 모델 파라메터를 로드하여 추론을 수행하겠습니다. 


## 1. Inference Graph 구축

In [ ]:
from PIL import Image
from matplotlib.pyplot import imshow
import os
import numpy as np
import glob
from os import listdir
from os.path import isfile, join
from utils import inference_utils as iu

In [ ]:
with open('./img_datasets/labels.txt') as f:
    l_lines = f.readlines()
    labels = [ line.replace('\n','') for line in l_lines ]

모델은 초기화 시에만 생성하여 메모리에 로드합니다. 매번 입력 데이터에 대해 모델을 재생성하는 것은 많은 지연 시간을 초래합니다.

In [ ]:
model_filepath='./model_result/inference_graph_frozen.pb'
model = iu.MobileNetInference(model_filepath, labels)

In [ ]:
all_tensors = model.get_all_tensors()
print(all_tensors[-1])

In [ ]:
model.get_input_node_info()

## 2. 샘플 데이터 추론

앞에서 정의한 클래스의 predict 메소드를 사용하여 간단하게 이미지 파일에 대한 추론 결과를 확인할 수 있습니다.

In [ ]:
test_img_path = './samples/*/*'
test_img_list = glob.glob(test_img_path)

아래 코드 셀을 여러 번 반복해 보세요 :) `CTRL+Enter` 단축키를 사용하시면 편리합니다.

In [ ]:
idx = np.random.randint(0,len(test_img_list))
test_image = test_img_list[idx]
print(test_image)
model.predict(test_image, img_size=224)

## 3. 배치 데이터 추론

여러분의 개인 랩탑/데스크탑이나 온프레미스에서 수행하는 방법과 동일하게 배치 데이터도 쉽게 추론이 가능합니다.  
본 예시에서는 테스트 데이터에 대해서 간단하게 배치 추론을 수행해 보고, 기본적인 평가 지표들인 ```Confusion Matrix```, ```AUROC(Area Under a ROC Curve)```, ```AUPRC(Area Under a Precision-Recall Curve)```를 확인해 보겠습니다.

In [ ]:
!pip install scikit-learn==0.23.1

In [ ]:
classnum_to_classname = {}
classname_to_classnum = {}
for class_name in labels:
    cls_split = class_name.split(':')
    classnum_to_classname[int(cls_split[0])] = cls_split[1]
    classname_to_classnum[cls_split[1]] = int(cls_split[0])

In [ ]:
test_img_path = './samples/*/*'
test_img_list = glob.glob(test_img_path)

In [ ]:
y_true_str = [img_list.split('/')[2] for img_list in test_img_list]
y_true = np.array([classname_to_classnum[s] for s in y_true_str])

아래 코드 셀에서 테스트 데이터셋에 대한 추론을 아래 절차로 수행합니다. 

- 정답값에 대한 One-hot encoding 변환 수행
- 배치 추론을 수행 후, 예측 score 및 결과(class) 리턴
- 예측 결과에 대한 One-hot encoding 변환 수행

In [ ]:
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(categories='auto', sparse=False)
num_classes = len(labels)

y_true_ohe = enc.fit_transform(y_true.reshape(-1, 1))
y_score, y_pred = iu.get_test_scores(model, test_img_list, '', num_classes)
y_pred_ohe = enc.transform(y_pred.reshape(-1,1))

y_pred_str = [classnum_to_classname[int(score)] for score in y_pred]

In [ ]:
y_pred[:10], y_pred_str[:10]

다중 클래스의 plotting에 필요한 컬러 테이블을 랜덤하게 생성합니다.

In [ ]:
import random
color_table = ["#"+''.join([random.choice('0123456789ABCDEF') for j in range(6)])
             for i in range(num_classes)]

### Confusion Matrix

In [ ]:
iu.plot_conf_mtx_multiclass(y_true, y_pred, labels)

### ROC Curve

In [ ]:
iu.plot_roc_curve_multiclass(y_true_ohe, y_score, num_classes, color_table)

### Precision-Recall Curve

In [ ]:
iu.plot_pr_curve_multiclass(y_true_ohe, y_score, num_classes, color_table)